In [ ]:
#| include: false
#| default_exp glycan_data
import warnings
warnings.filterwarnings("ignore")
from nbdev.showdoc import show_doc
from IPython.display import HTML
import copy
#%load_ext autoreload
#%autoreload 2

In [ ]:
#| export
from glycowork.glycan_data.loader import *
from glycowork.glycan_data.loader import glycan_binding, df_glycan, df_species
from glycowork.glycan_data.stats import *
from glycowork.glycan_data.data_entry import *

`glycan_data` contains several helper functions for glycan data loading and data entry. Helper functions for data loading and data objects are in `loader` and include:

- `unwrap` flattens nested lists
- `find_nth` returns the n-th occurrence of a motif
- `df_species` loaded file for all glycans with species information, one row per glycan-species combination
- `df_glycan` loaded file containing all unique glycans with lots of meta-information
- `glycan_motifs` literature-known glycan motifs together with their sequence
- `glycan_binding` Z-score data for 1,392 lectins binding to glycans; every row one protein, glycans are columns
- `lib` library of unique monosaccharides + linkages in stored datasets; generated with `get_lib` & stored pre-computed

Further, the `stats` module contains many helper functions for statistics and data normalization.

## loader

In [ ]:
show_doc(unwrap)

---

### unwrap

>      unwrap (nested_list)

converts a nested list into a flat list

In [ ]:
unwrap([[1,2], [3,4]])

[1, 2, 3, 4]

In [ ]:
show_doc(find_nth)

---

### find_nth

>      find_nth (haystack, needle, n)

finds n-th instance of motif

| Arguments:
| :-
| haystack (string): string to search for motif
| needle (string): motif
| n (int): n-th occurrence in string (not zero-indexed)

| Returns:
| :-
| Returns starting index of n-th occurrence in string

In [ ]:
find_nth('This is as good as it gets', 'as', 2)

16

In [ ]:
show_doc(find_nth_reverse)

---

### find_nth_reverse

>      find_nth_reverse (string, substring, n, ignore_branches=False)

In [ ]:
find_nth_reverse("Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc", "Gal", 0)

38

In [ ]:
show_doc(replace_every_second)

---

### replace_every_second

>      replace_every_second (string, old_char, new_char)

function to replace every second occurrence of old_char in string with new_char

| Arguments:
| :-
| string (string): a string
| old_char (string): a string character to be replaced (every second occurrence)
| new_char (string): the string character to replace old_char with

| Returns:
| :-                           
| Returns string with replaced characters

In [ ]:
replace_every_second("Neu5Ac(a2-3(Gal(b1-4(Glc", '(', ')')

'Neu5Ac(a2-3)Gal(b1-4)Glc'

In [ ]:
show_doc(multireplace)

---

### multireplace

>      multireplace (string, remove_dic)

Replaces all occurences of items in a set with a given string

| Arguments:
| :-
| string (str): string to perform replacements on
| remove_dic (set): dict of form to_replace:replace_with

| Returns:
| :-
| (str) modified string

In [ ]:
multireplace("Neu5Ac[a2-3]Gal", {'[': '(', ']': ')'})

'Neu5Ac(a2-3)Gal'

In [ ]:
show_doc(build_custom_df)

---

### build_custom_df

>      build_custom_df (df, kind='df_species')

creates custom df from df_glycan

| Arguments:
| :-
| df (dataframe): df_glycan / sugarbase
| kind (string): whether to create 'df_species', 'df_tissue', or 'df_disease' from df_glycan; default:df_species

| Returns:
| :-
| Returns custom df in the form of one glycan - species/tissue/disease association per row

In [ ]:
build_custom_df(df_glycan, kind = 'df_tissue')

glycan  tissue_sample  \
0      Fuc(a1-2)Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1...  2A3_cell_line   
1      Fuc(a1-?)[Gal(b1-4)]GlcNAc(b1-2)Man(a1-3)[Man(...  2A3_cell_line   
2      Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Gal(b1-4)GlcNAc...  2A3_cell_line   
3      Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Gal(b1-4)GlcNAc...  2A3_cell_line   
4      Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1-3)[Man(a...  2A3_cell_line   
...                                                  ...            ...   
16711  Neu5Ac(a2-?)Gal(b1-4)GlcNAc(b1-?)[Gal(b1-4)Glc...   zone_of_skin   
16712        Neu5AcOAc(a2-8)Neu5Ac(a2-3)Gal(b1-4)Glc1Cer   zone_of_skin   
16713  {Fuc(a1-?)}Neu5Ac(a2-?)Gal(b1-4)GlcNAc(b1-2)Ma...   zone_of_skin   
16714  {Man(a1-2)}Man(a1-2)Man(a1-3)[Man(a1-3)[Man(a1...   zone_of_skin   
16715  {Man(a1-2)}{Man(a1-2)}Man(a1-2)Man(a1-3)[Man(a...   zone_of_skin   

       tissue_species              tissue_id  \
0        Homo_sapiens  cellosaurus:CVCL_0D71   
1        Homo_sapiens  cellosaurus:CVCL_0D71   
2        Homo_sapiens  cellosaurus:CVCL_0D71   
3        Homo_sapiens  cellosaurus:CVCL_0D71   
4        Homo_sapiens  cellosaurus:CVCL_0D71   
...               ...                    ...   
16711    Homo_sapiens         UBERON:0000014   
16712  Xiphophorus_sp         UBERON:0000014   
16713    Homo_sapiens         UBERON:0000014   
16714    Homo_sapiens         UBERON:0000014   
16715    Homo_sapiens         UBERON:0000014   

                                              tissue_ref  
0              https://pubmed.ncbi.nlm.nih.gov/36289103/  
1              https://pubmed.ncbi.nlm.nih.gov/36289103/  
2              https://pubmed.ncbi.nlm.nih.gov/36289103/  
3              https://pubmed.ncbi.nlm.nih.gov/36289103/  
4              https://pubmed.ncbi.nlm.nih.gov/36289103/  
...                                                  ...  
16711  http://www.functionalglycomics.org/glycomics/S...  
16712           https://pubmed.ncbi.nlm.nih.gov/3370642/  
16713  http://www.functionalglycomics.org/glycomics/S...  
16714  http://www.functionalglycomics.org/glycomics/S...  
16715  http://www.functionalglycomics.org/glycomics/S...  

[16716 rows x 5 columns]

`df_species`

In [ ]:
#| echo: false
df_species2 = copy.deepcopy(df_species)
df_species2.set_index("glycan", inplace = True)
df_species2.head().style.set_properties(**{'font-size': '11pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

`df_glycan`

In [ ]:
#| echo: false
df_glycan2 = copy.deepcopy(df_glycan)
df_glycan2.set_index("glycan", inplace = True)
df_glycan2.head().style.set_properties(**{'font-size': '11pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

`glycan_binding`

In [ ]:
#| echo: false
glycan_binding.head().style.set_properties(**{'font-size': '11pt', 'font-family': 'Helvetica','border-collapse': 'collapse','border': '1px solid black'})

## stats

In [ ]:
show_doc(get_BF)

---

### get_BF

>      get_BF (n, p, z=False, method='robust', upper=10)

Transforms a p-value into Jeffreys' approximate Bayes factor (BF)

| Arguments:
| :-
| n (int): Sample size.
| p (float): The p-value.
| z (bool): True if the p-value is based on a z-statistic, False if t-statistic; default:False
| method (str): Method used for the choice of 'b'. Options are "JAB", "min", "robust", "balanced"; default:'robust'
| upper (float): The upper limit for the range of realistic effect sizes. Only relevant when method="balanced"; default:10

| Returns:
| :-
| float: A numeric value for the BF in favour of H1.

In [ ]:
get_BF(6, 0.05)

30.1559980528856

In [ ]:
show_doc(get_alphaN)

---

### get_alphaN

>      get_alphaN (n, BF=3, method='robust', upper=10)

Set the alpha level based on sample size via Bayesian-Adaptive Alpha Adjustment.

| Arguments:
| :-
| n (int): Sample size.
| BF (float): Bayes factor you would like to match; default:3
| method (str): Method used for the choice of 'b'. Options are "JAB", "min", "robust", "balanced"; default:"robust"
| upper (float): The upper limit for the range of realistic effect sizes. Only relevant when method="balanced"; default:10

| Returns:
| :-
| float: Numeric alpha level required to achieve the desired level of evidence.

In [ ]:
get_alphaN(6)

You're working with an alpha of 0.07862467893233027 that has been adjusted for your sample size of 6.


0.07862467893233027

In [ ]:
show_doc(TST_grouped_benjamini_hochberg)

---

### TST_grouped_benjamini_hochberg

>      TST_grouped_benjamini_hochberg (identifiers_grouped, p_values_grouped,
>                                      alpha)

perform the two-stage adaptive Benjamini-Hochberg procedure for multiple testing correction

| Arguments:
| :-
| identifiers_grouped (dict): dictionary of group : list of glycans
| p_values_grouped (dict): dictionary of group : list of p-values
| alpha (float): significance threshold for testing

| Returns:
| :-
| Returns dictionaries of glycan : corrected p-value and glycan : significant?

In [ ]:
grouped_glycans = {'core2': ['Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc'], 'core1': ['Gal(b1-3)[Neu5Ac(a2-6)]GalNAc', 'GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc', 'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)]GalNAc']}
grouped_pvals = {'core2': [0.6142626731590599, 0.7070153617674607, 0.019898944554207577, 0.8026651973502792, 0.015092570600834338, 0.0023818885119090837, 0.5611180077391343, 0.005416655300198711], 'core1': [0.38054124309577675, 0.0003019282861611496, 0.03584424395333232, 0.010311562841413265, 0.7501582090523815]}
alpha = 0.054572814701647765
TST_grouped_benjamini_hochberg(grouped_glycans, grouped_pvals, alpha)

({'Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]GalNAc': 0.039797889108415155,
  'Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 0.039797889108415155,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc': 0.01905510809527267,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]GalNAc': 0.8026651973502792,
  'Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc': 0.021666621200794843,
  'Gal(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.4756765538697209,
  'GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.001509641430805748,
  'Neu5Ac(a2-3)Gal(b1-3)GalNAc': 0.059740406588887206,
  'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc': 0.02577890710353316,
  'Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)]GalNAc': 0.7501582090523815},
 {'Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc': 

## data_entry

Helper functions for entering data are contained in `data_entry` and include:

- `check_presence` tests whether the entered glycan is already in database

In [ ]:
show_doc(check_presence)

---

### check_presence

>      check_presence (glycan, df, colname='glycan', name=None, rank='Species',
>                      fast=False)

checks whether glycan (of that species) is already present in dataset

| Arguments:
| :-
| glycan (string): IUPAC-condensed glycan sequence
| df (dataframe): glycan dataframe where glycans are under colname and ideally taxonomic labels are columns
| name (string): name of the species (etc.) of interest
| rank (string): column name for filtering; default: species
| fast (bool): True uses precomputed glycan graphs, only use if df has column 'graph' with glycan graphs

| Returns:
| :-
| Returns text output regarding whether the glycan is already in df

In [ ]:
print("Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
check_presence('Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species)

print("Check presence of Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
check_presence('Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species)

print("Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc in the species Danielus Bojarum")
check_presence('Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc', df_species,
               name = 'Danielus Bojarum')

Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc
Glycan already in dataset.
Check presence of Fuc(b1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc
It's your lucky day, this glycan is new!
Check presence of Man(a1-3)[Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc in the species Danielus Bojarum
This is the best: Danielus_Bojarum is not in dataset
It's your lucky day, this glycan is new!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()